In [1]:
# URL = 'http://localhost:3000'
URL = 'https://e-catalogue.abcdavid.top'

In [2]:
import requests
from requests_toolbelt.multipart.encoder import MultipartEncoder

In [3]:
def FileExists(filename):
    try:
        with open(filename, 'r') as f:
            return True
    except FileNotFoundError:
        return False

In [4]:
import enum

class RequestType(enum.Enum):
    GET = 0
    POST = 1
    PUT = 2
    DELETE = 3
    
class RequestBuilder:
    def __init__(self, url):
        self.url = url
        self.headers = {'Accept': 'application/json'}
        self.query = {}
        self.fields = {}
        self.token = None
        self.isMultipart = False
    
    def SetToken(self, token):
        self.token = token
        return self
        
    def AddHeader(self, key, value):
        self.headers[key] = str(value)
        return self
    
    def AddBody(self, key, value):
        self.fields[key] = str(value)
        return self
        
    def AddQuery(self, key, value):
        self.query[key] = str(value)
        return self
    
    def GetFile(self, path):
        if not FileExists(path):
            raise FileNotFoundError(path)
        
        filename = path.split('/')[-1]
        
        filetype = filename.split('.')[-1].lower()
        mimetype = None
        if filetype == 'jpg' or filetype == 'jpeg':
            mimetype = 'image/jpeg'
            
        if filetype == 'png':
            mimetype = 'image/png'
        
        if not mimetype:
            raise Exception('Unsupported file type')
        
        return (filename, open(path, 'rb'), mimetype)
    
    def AddFile(self, fieldname, path):
        # self.files.append((fieldname, self.GetFile(path)))
        self.fields[fieldname] = self.GetFile(path)
        self.isMultipart = True
        return self
        
    def Build(self, request_type):
        new_headers = self.headers.copy()
        if (self.token):
            new_headers['Authorization'] = 'Bearer ' + self.token
 
        if self.isMultipart and request_type in [RequestType.POST, RequestType.PUT]:
            encoder = MultipartEncoder(fields=self.fields)
            new_headers['Content-Type'] = encoder.content_type
            if request_type == RequestType.POST:
                return requests.post(self.url, headers=new_headers, params=self.query, data=encoder)
            elif request_type == RequestType.PUT:
                return requests.put(self.url, headers=new_headers, params=self.query, data=encoder)
            
        if request_type == RequestType.GET:
            return requests.get(self.url, headers=new_headers, params=self.query)
        elif request_type == RequestType.POST:
            return requests.post(self.url, headers=new_headers, params=self.query, data=self.fields)
        elif request_type == RequestType.PUT:
            return requests.put(self.url, headers=new_headers, params=self.query, data=self.fields)
        elif request_type == RequestType.DELETE:
            return requests.delete(self.url, headers=new_headers, params=self.query)
        raise Exception('Unsupported request type')
    
    def GET(self):
        return self.Build(RequestType.GET)
    
    def POST(self):
        return self.Build(RequestType.POST)
    
    def PUT(self):
        return self.Build(RequestType.PUT)
    
    def DELETE(self):
        return self.Build(RequestType.DELETE)
        

In [5]:
def login(username, password):

    res = (RequestBuilder(URL+'/auth/login')
           .AddBody('username', username)
           .AddBody('password', password)
           .POST())
    token = None
    if (res.status_code in [200, 201]):
        print(f'{username} login success')
        token = res.json()['access_token']
    else:
        print(f'{username} login failed')
        if (res.json()['message'] == 'User not found'):
            # raise Exception('User not found')
            return None
        else:
            print(res.json())
            exit(1)
    return token

In [6]:
class TokenManager:
    __STORE_TOKEN = None
    __STORE_USERNAME = 'spino'
    __STORE_PASSWORD = '123456'
    __STORE_ID = None
    
    @staticmethod
    def __init_store():
        TokenManager.__STORE_TOKEN = login(TokenManager.__STORE_USERNAME, TokenManager.__STORE_PASSWORD)
        if not TokenManager.__STORE_TOKEN:
            CreateUserRequest = (RequestBuilder(URL+'/users/createuser')
                                .AddBody('username', TokenManager.__STORE_USERNAME)
                                .AddBody('password', TokenManager.__STORE_PASSWORD)
                                .AddBody('email', TokenManager.__STORE_USERNAME + '@gmail.com')
                                .POST())
            if (CreateUserRequest.status_code not in [200, 201]):
                print('Create account failed')
            TokenManager.__STORE_TOKEN = login(TokenManager.__STORE_USERNAME, TokenManager.__STORE_PASSWORD)
                
        RegisterStoreRequest = (RequestBuilder(URL+'/product/store')
                                .AddBody('name', 'Abcdavid Store')
                                .AddBody('description', 'Store of Abcdavid')
                                .AddBody('address', '0')
                                .AddFile('logo', './CategoryImages/Boy-Accessories.png')
                                .AddHeader('Content-Type', 'multipart/form-data; charset=utf-8')
                                .SetToken(TokenManager.__STORE_TOKEN)
                                .POST())
        if (RegisterStoreRequest.status_code not in [200, 201]):
            print('Register store failed')
            exit(1)
        TokenManager.__STORE_ID = RegisterStoreRequest.json()['id']
    
    @staticmethod
    def StoreId():
        if not TokenManager.__STORE_ID:
            TokenManager.__init_store()
        return TokenManager.__STORE_ID
    
    @staticmethod
    def StoreToken():
        if not TokenManager.__STORE_TOKEN:
            TokenManager.__init_store()
        return TokenManager.__STORE_TOKEN

In [7]:
def calc_indent(numb: int) -> str:
    if numb < 2:
        return ""
    binary = bin(numb)[3:]
    ret = map(lambda x: "│  " if x == "1" else "   ", binary[:-1])
    return "".join(ret) + ( "└──" if binary[-1] == "0" else "├──")

class Category:
    def __init__(self, name, description, image):
        self.id = None
        self.name = name
        self.description = description if description else None
        self.children = []
        self.image = image
        self.same_name = True
        self.same_description = True
        
    def from_json(json):
        cat = Category(json['name'], json['description'], json['image'])
        cat.id = json['id']
        for child in json['children']:
            cat.children.append(Category.from_json(child))
        return cat
        
    def add_child(self, child):
        self.children.append(child)
    
    def add_child(self, name, description, image):
        self.children.append(Category(name, description, image))
        
    def add_children(self, children):
        self.children.extend(map(lambda x: 
            Category(x[0],x[1],x[2])
            if isinstance(x, tuple) 
            else Category(x, None, None)
            , children))
        
    def image_exists(self):
        if self.image:
            return FileExists('./CategoryImages/' + self.image)
        return False
        
    def __str__(self, indent=1):
        ret = calc_indent(indent) + self.__repr__() + "\n"
        for child in self.children[:-1]:
            ret += child.__str__(indent*2+1)
        if len(self.children) > 0:
            ret += self.children[-1].__str__(indent*2)
        return ret
    
    def __eq__(self, __value: object) -> bool:
        if isinstance(__value, Category):
            return self.name == __value.name
        else:
            return False
        
    def match(self, target: object):
        if not self == target:
            return
        self.id = target.id
        if not self.description and target.description:
            self.description = target.description
        if self.name != target.name:
            self.same_name = False
        if self.description != target.description:
            self.same_description = False
        for child in self.children:
            for target_child in target.children:
                if child == target_child:
                    child.match(target_child)
        
    def short_desc(self):
        if self.description:
            return self.description[:20] + "..." if len(self.description) > 20 else self.description
        else:
            return ""
    
    def __repr__(self):
        return f'[{self.id if self.id else "?"}] {"!" if not self.same_name else ""}{self.name} - {"!" if not self.same_description else ""}"{self.short_desc()}" {"!" if not self.image_exists() else ""}[{self.image if self.image else "None"}]'

    def get_child(self, name):
        for child in self.children:
            if child.name == name:
                return child
        return None

    def get_name_by_id(self, id):
        if self.id == id:
            return self.name
        for child in self.children:
            name = child.get_name_by_id(id)
            if name:
                return self.name + '/' + name
        return None

    def __getitem__(self, key):
        if hasattr(self, 'children') and isinstance(self.children, (list, dict)):
            return self.get_child(key)
        else:
            raise AttributeError("The 'children' attribute does not exist or is not subscriptable.")

In [8]:
class ServerCategory:
    
    __categories = {}
    __fetch = False
    
    @staticmethod
    def get_categories(cls):
        res = (RequestBuilder(URL+'/product/category/all')
               .GET())
        if res.status_code != 200:
            print(res).json()
            exit()
        for cat in res.json():
            cls.__categories[cat['name']] = Category.from_json(cat)
        return cls.__categories
    
    @staticmethod
    def __get_categories(cls):
        if not cls.__fetch:
            cls.__categories = cls.get_categories(cls)
            # cls.__fetch = True
        return cls.__categories
    
    @staticmethod
    def all():
        cats = []
        for cat in ServerCategory.__get_categories(ServerCategory).values():
            cats.append(cat)
        return cats
    
    def __class_getitem__(cls, key):
        return cls.__get_categories(cls).get(key)
    

In [9]:
print(ServerCategory.all())
print(ServerCategory['Men'])

[[1] Men - "" ![None], [30] Women - "" ![None], [64] Boy - "" ![None], [81] Girl - "" ![None]]
[1] Men - "" ![None]
├──[2] Clothing - "" ![c3be579b-a8d2-11ee-94f9-c239f7a625f9.png]
│  ├──[3] Coats - "" ![None]
│  ├──[4] Cardigans and sweaters - "" ![None]
│  ├──[5] Jackets and overshirts - "" ![None]
│  ├──[6] Trousers - "" ![None]
│  ├──[7] Shirts - "" ![None]
│  ├──[8] Jeans - "" ![None]
│  ├──[9] T-Shirts - "" ![None]
│  ├──[10] Polos - "" ![None]
│  ├──[11] Blazers - "" ![None]
│  └──[12] Underwear - "" ![None]
├──[13] Suits - "" ![c4122a4c-a8d2-11ee-94f9-c239f7a625f9.png]
│  ├──[14] Suits - "" ![None]
│  ├──[15] Blazers - "" ![None]
│  ├──[16] Trousers - "" ![None]
│  ├──[17] Waistcoats - "" ![None]
│  ├──[18] Shirts - "" ![None]
│  └──[19] Accessories - "" ![None]
└──[20] Shoes and accessories - "" ![c462e935-a8d2-11ee-94f9-c239f7a625f9.png]
   ├──[21] Shoes - "" ![None]
   ├──[22] Backpacks and bags - "" ![None]
   ├──[23] Belts and braces - "" ![None]
   ├──[24] Scarvers, caps 

In [10]:
print(ServerCategory['Women'])

[30] Women - "" ![None]
├──[31] Clothing - "" ![cc40db55-a8d2-11ee-94f9-c239f7a625f9.png]
│  ├──[32] Coats - "" ![None]
│  ├──[33] Dresses and jumpsuits - "" ![None]
│  ├──[34] Jackets and Suit Jackets - "" ![None]
│  ├──[35] Sweaters and cardigans - "" ![None]
│  ├──[36] Shirts - "" ![None]
│  ├──[37] Leather - "" ![None]
│  ├──[38] T-Shirts and tops - "" ![None]
│  ├──[39] Sweatshirts - "" ![None]
│  ├──[40] Trousers - "" ![None]
│  ├──[41] Skirts - "" ![None]
│  ├──[42] Jeans - "" ![None]
│  └──[43] Pikinis and swimsuits - "" ![None]
├──[44] Shoes and Accessories - "" ![ce8be591-a8d2-11ee-94f9-c239f7a625f9.png]
│  ├──[45] Shoes - "" ![None]
│  ├──[46] Bags - "" ![None]
│  ├──[47] Jewellery - "" ![None]
│  ├──[48] Wallets and cases - "" ![None]
│  ├──[49] Belts - "" ![None]
│  ├──[50] Sunglasses - "" ![None]
│  ├──[51] Scarvers and foulards - "" ![None]
│  ├──[52] Caps and glovers - "" ![None]
│  └──[53] Fragrances - "" ![None]
└──[54] Plus Sizes - "" ![d077fdf9-a8d2-11ee-94f9-c239f7

Add Product

In [11]:
def findCategoryIdByName(name):
    names = name.split('/')
    try:
        if len(names) == 1:
            return ServerCategory[names[0]].id
        elif len(names) == 2:
            return ServerCategory[names[0]][names[1]].id
        elif len(names) == 3:
            return ServerCategory[names[0]][names[1]][names[2]].id
        else:
            raise Exception("Invalid category name")
    except AttributeError as e:
        if "'NoneType' object has no attribute 'id'" in e.args:
            return None
        else:
            raise e

def getCategoryNameById(id):
    for cat in ServerCategory.all():
        name = cat.get_name_by_id(id)
        if name:
            return name
    return None

class ProductVariant:
    def __init__(self, product, color, size, price, quantity, image):
        self.id = None
        self.color = color
        self.size = size
        self.price = price
        self.quantity = quantity
        self.image = image
    
    def __repr__(self):
        return f'{self.color} - {self.size} - {self.price} - {self.quantity} - {self.image}'
    
    def image_exists(self):
        if self.image:
            return FileExists('./ProductImages/' + self.image)
        return False
    
    def update(self, productId: int):
        print(f'Updating variant {self} with product id {productId}')
        request = RequestBuilder(URL+'/product/variant').SetToken(TokenManager.StoreToken())

        request.AddQuery('product', str(productId))
        if self.color:
            request.AddQuery('color', self.color)
        if self.size:
            request.AddQuery('size', self.size)
        if self.price:
            request.AddBody('price', str(self.price))
        if self.quantity:
            request.AddBody('quantity', self.quantity)
        if self.image:
            request.AddFile('image', './ProductImages/' + self.image)
        res = request.POST()
        if res.status_code not in [200, 201]:
            print(f'Update product variant failed [{self.product}]')
            print(res.json())
            # raise Exception(f'[{res.status_code}] Failed to update product variant {self.product}')
        self.id = res.json()['id']
        print (f'[{self.id}] Updated product[{productId}] - {self.color} - {self.size} - {self.price} - {self.quantity}')        

class Product:
    def __init__(self, name, description, category, image):
        self.name = name
        self.description = description
        self.same_description = False
        self.category = category
        self.image = image
        self.variant = []
        
        self.id = None
        cat = findCategoryIdByName(self.category)
        if cat:
            self.category_id = cat
        else:
            raise Exception(f'Category {self.category} not found')
        # self.fetch()
        
    @staticmethod
    def all():
        return (RequestBuilder(URL+'/product/store')
                .AddQuery('id', TokenManager.StoreId())
                .GET()).json()['products']
        
    def fetch(self):
        products = Product.all()
        product = next((product for product in products if product['name'] == self.name and product['category']['id'] == self.category_id), None)
        if product:
            self.id = product['id']
            self.same_description = self.description == product['description']
    
    def add_variant(self, color, size, price, quantity, image):
        self.variant.append(ProductVariant(self.id, color, size, price, quantity, image))
        
    def add_variant_without_image(self, color, size, price, quantity):
        self.variant.append(ProductVariant(self.id, color, size, price, quantity, self.image))
    
    def __repr__(self):
        return f'[{self.id if self.id else "?"}] {"!" if not self.same_description else ""}{self.name} - "{self.description}" {"!" if not self.image_exists() else ""}[{self.image if self.image else "None"}]'
    
    def __str__(self) -> str:
        ret = self.__repr__() + "\n"
        for variant in self.variant[:-1]:
            ret += "├──" + variant.__repr__() + "\n"
        if len(self.variant) > 0:
            ret += "└──" + self.variant[-1].__repr__()
        return ret
    
    def image_exists(self):
        if self.image:
            return FileExists('./ProductImages/' + self.image)
        return False
    
    def update(self):
        if not self.id:
            self.__add()
        else:
            request = RequestBuilder(URL+'/product').SetToken(TokenManager.StoreToken()).AddQuery('id', self.id)
            if self.name:
                request.AddBody('name', self.name)
            if self.description:
                request.AddBody('description', self.description)
            if self.category_id:
                request.AddBody('category', self.category_id)
            if self.image:
                request.AddFile('image', './ProductImages/' + self.image)
            res = request.PUT()
            if res.status_code not in [200, 201]:
                print(f'Update product failed [{self.name}]')
                print(res.json())
                # raise Exception(f'[{res.status_code}] Failed to update product {self.name}')
            print (f'[{res.status_code}] Updated {self.name} - "{self.description}"')

        for variant in self.variant:
            print(f'Updating variant {variant} with product id {self.id}')
            variant.update(self.id)
            
    def __add(self):
        if self.id:
            print(f'Product {self.name} already exists')
            return
        if not self.name:
            raise Exception('Product name is required') 
        if not self.category_id:
            raise Exception('Product category is required')
        if not self.image or not self.image_exists():
            raise Exception('Product image is required')
        request = (RequestBuilder(URL+'/product/')
                   .SetToken(TokenManager.StoreToken())
                   .AddBody('name', self.name)
                   .AddBody('description', self.description)
                   .AddBody('category', self.category_id)
                   .AddFile('image', './ProductImages/' + self.image)
                   .POST())
        if request.status_code not in [200, 201]:
            print(f'Add product failed [{self.name}]')
            print(request.json())
            # raise Exception(f'[{request.status_code}] Failed to add product {self.name}')
        self.id = request.json()['id']
        print('Added product ' + self.name + ' with id ' + str(self.id))
    
    
    @staticmethod
    def from_json(json):
        return Product(json['name'], json['description'], json['category'], json['image'])

In [12]:
getCategoryNameById(28)

'Men/Shoes and accessories/Tailoring Accessories'

In [13]:
findCategoryIdByName('Men/Shoes and accessories/Shoes')

21

In [14]:
import pandas as pd
import os

data_folder = "./Data/Product/"  # replace with your actual folder path
files = os.listdir(data_folder)

dfs = []
for file in files:
    if file.endswith('.csv'):
        product_dataframe = pd.read_csv(os.path.join(data_folder, file))
        dfs.append(product_dataframe)

pdf = pd.concat(dfs, ignore_index=True).sort_values('id', ascending=True)

In [15]:
from enum import Enum

class Color(Enum):
    WHITES = 'Whites'
    BLACKS = 'Blacks'
    GREYS = 'Greys'
    BEIGES = 'Beiges'
    BROWNS = 'Browns'
    REDS = 'Reds'
    GREENS = 'Greens'
    BLUES = 'Blues'
    PURPLES = 'Purples'
    YELLOWS = 'Yellows'
    PINKS = 'Pinks'
    ORANGES = 'Oranges'
    
similar_color = {
    Color.WHITES.value: ['Silver'],
    Color.BLUES.value: ['Navy'],
    Color.BEIGES.value: ['Khaki', 'Nude', 'Ecru', 'Sand'],
    Color.BROWNS.value: ['Caramel', 'Chocolate', 'Coffee'],
    Color.REDS.value: ['Maroon', 'Burgundy'],
    Color.YELLOWS.value: ['Curry'],
    Color.PURPLES.value: ['Plum'],
    Color.PINKS.value: ['Fuchsia']
}


def parseColor(color_name: str) -> Color:
    for color in Color:
        if color.value[:-1].lower() in color_name.lower():
            return color
        if not similar_color.get(color.value):
            continue
        if len(similar_color[color.value]) <= 0:
            continue
        for sc in similar_color[color.value]:
            if sc.lower() in color_name.lower():
                return color
    return None
   
def canParseColor(color_name: str) -> bool:
    return parseColor(color_name) is not None

colors = pdf[pdf['name'].apply(lambda x: not canParseColor(x))]['name'].unique()
colors

array([], dtype=object)

In [16]:
class Size(Enum):
    XS = 'XS'
    S = 'S'
    M = 'M'
    L = 'L'
    XL = 'XL'
    XXL = 'XXL'

def variate_price(price: int, size: Size) -> int:
    price_fraction = round(price * 5 / 100, -2)
    if size == Size.XS:
        return price - price_fraction * 3
    if size == Size.S:
        return price - price_fraction * 2
    if size == Size.M:
        return price - price_fraction
    if size == Size.L:
        return price + price_fraction
    if size == Size.XL:
        return price + price_fraction * 2
    if size == Size.XXL:
        return price + price_fraction * 3

In [17]:
import pandas as pd
import os

data_folder = "./Data/Category/"  # replace with your actual folder path
files = os.listdir(data_folder)

dfs = []
for file in files:
    if file.endswith('.csv'):
        product_dataframe = pd.read_csv(os.path.join(data_folder, file))
        dfs.append(product_dataframe)

all_product_dataframe = pd.concat(dfs, ignore_index=True).sort_values('id', ascending=True)

# product_dataframe = pd.concat(dfs, ignore_index=True).sort_values('id', ascending=True)
product_dataframe = all_product_dataframe[all_product_dataframe['id'] % 2 == 0]

In [18]:
i = 0
for row in product_dataframe.values.tolist():
    i += 1
    if i > 5:
        break
    try:
        print('CategoryID:',findCategoryIdByName(row[2]))
    except Exception as e:
        print('CategoryID invalid:', row[2], '--'*50)
    print(row)

CategoryID: 3
[2, 'Long recycled wool coat', 'Men/Clothing/Coats', 5499000]
CategoryID: 3
[4, 'Long recycled wool coat', 'Men/Clothing/Coats', 5499000]
CategoryID: 3
[6, 'Classic water-repellent trench coat', 'Men/Clothing/Coats', 4999000]
CategoryID: 3
[8, 'Reversible water-repellent quilted parka', 'Men/Clothing/Coats', 5999000]
CategoryID: 3
[10, 'Wool funnel neck coat', 'Men/Clothing/Coats', 5499000]


In [19]:
import random

product_data_folder = "./Data/Product/"

Products = []
for row in product_dataframe.values.tolist():
    path = os.path.join(product_data_folder, str(row[0]) + '.csv')
    pdf = pd.read_csv(path)
    product = Product(
        name=row[1],
        description=row[1],
        category=row[2],
        image=str(pdf['image_id'][0]) + '.png'
    )
    price = row[3]
    
    for variant in pdf.values.tolist():
        random_sizes = random.sample([Size.XS, Size.S, Size.M, Size.L, Size.XL, Size.XXL], random.randint(1, 6))
        for size in random_sizes:
            product.add_variant(
                color = parseColor(variant[1]).value,
                size = size.value,
                price = round(variate_price(price, size) / 23000),
                quantity = random.randint(100, 1000),
                image = str(variant[0]) + '.png'
            )
    Products.append(product)

In [20]:
for product in Products:
    print(product)
    product.update()

[?] !Long recycled wool coat - "Long recycled wool coat" [2.png]
├──Browns - L - 251 - 356 - 2.png
├──Browns - XS - 203 - 428 - 2.png
├──Browns - S - 215 - 115 - 2.png
├──Browns - M - 227 - 546 - 2.png
├──Blues - S - 215 - 160 - 3.png
├──Blues - XS - 203 - 919 - 3.png
├──Blues - M - 227 - 318 - 3.png
├──Blues - XL - 263 - 404 - 3.png
├──Blues - XXL - 275 - 979 - 3.png
├──Blacks - XL - 263 - 353 - 4.png
├──Greys - XS - 203 - 440 - 5.png
├──Greys - XXL - 275 - 171 - 5.png
├──Greys - L - 251 - 336 - 5.png
├──Greys - M - 227 - 696 - 5.png
├──Greys - XL - 263 - 139 - 5.png
└──Greys - S - 215 - 223 - 5.png
spino login success
Added product Long recycled wool coat with id 188
Updating variant Browns - L - 251 - 356 - 2.png with product id 188
Updating variant Browns - L - 251 - 356 - 2.png with product id 188
[806] Updated product[188] - Browns - L - 251 - 356
Updating variant Browns - XS - 203 - 428 - 2.png with product id 188
Updating variant Browns - XS - 203 - 428 - 2.png with product id 

In [21]:
len(Products)

76

In [22]:
product_dataframe[product_dataframe['id'] % 2 == 1]
a = product_dataframe[product_dataframe['id'] % 2 == 1].values.tolist()
a

[]

In [23]:
Products = []
